In [1]:
import os
import numpy as np
import pandas as pd 
#import modin.pandas as pd
from tqdm import tqdm
import re
import datetime
from p_tqdm import p_map
from functools import partial
import gc

In [2]:
! pip -V

pip 21.3.1 from /home/orzwang/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
df_mentor = pd.read_csv("mentor_list_verified.csv")
df_mentor.head()

,project_name,mentor_alias,mentor_full,ascii
0,abdera,rooneg,Garrett Rooney,Garrett Rooney
1,abdera,pquerna,Paul Querna,Paul Querna
2,accumulo,bimargulies,Benson Margulies,Benson Margulies
3,accumulo,adc,Alan D. Cabrera,Alan D. Cabrera
4,accumulo,berndf,Bernd Fondermann,Bernd Fondermann


In [23]:
time_resolution = '1'
lk_path = '/mnt/data0/lkyin/'
c_path = '../network_data'+str(time_resolution)+'/commits/'
e_path = '../network_data'+str(time_resolution)+'/emails/'
mix_path = '../network_data'+str(time_resolution)+'/mix/'

In [20]:
# ------------------- processing commits ---------------------- 
print('reading commits...')
df = pd.read_csv(lk_path+'commits_final.csv')
print('grouping by project...')
df = dict(tuple(df.groupby(df['project_name'])))
to_path = '../byauthor_data/commits/'
if not os.path.exists(to_path):
	os.makedirs(to_path)

reading commits...


/home/orzwang/.pyenv/versions/3.8.10/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


grouping by project...


In [9]:
df.keys()

dict_keys(['abdera', 'accumulo', 'ace', 'activemq', 'adffaces', 'agila', 'airavata', 'airflow', 'allura', 'alois', 'amaterasu', 'ambari', 'ant', 'any23', 'apex', 'apisix', 'ariatosca', 'aries', 'asterixdb', 'atlas', 'aurora', 'awf', 'batchee', 'beam', 'beehive', 'bigtop', 'bloodhound', 'bluesky', 'blur', 'bookkeeper', 'brooklyn', 'buildr', 'bval', 'calcite', 'carbondata', 'cassandra', 'cayenne', 'celix', 'chemistry', 'chukwa', 'clerezza', 'click', 'climate', 'cloudstack', 'cmda', 'commons', 'commonsrdf', 'composer', 'concerted', 'cordova', 'corinthia', 'couchdb', 'crunch', 'ctakes', 'curator', 'cxf', 'daffodil', 'datafu', 'datasketches', 'deltacloud', 'deltaspike', 'derby', 'devicemap', 'directmemory', 'directory', 'dolphinscheduler', 'drill', 'droids', 'druid', 'dubbo', 'eagle', 'echarts', 'edgent', 'empire', 'esme', 'etch', 'falcon', 'felix', 'fineract', 'flex', 'flink', 'flume', 'fluo', 'freemarker', 'ftpserver', 'gearpump', 'geode', 'giraph', 'gobblin', 'gora', 'gossip', 'graffito'

In [11]:
df['abdera'].head(10)

,project_name,list_name,date,month,message_id,sender_name,sender_email,author_name,author_email,file_name,loc,ref_or_sha,subject,commit_type,author_full_name,is_bot,is_coding,dealised_author_full_name
4170335,abdera,commits,2007-01-05 17:58:47,7,<20070105175847.C76E31A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/core/src/main/java...,37.0,493105.0,svn commit: r493105 - in /incubator/abdera/jav...,svn,james m. snell,False,True,James Snell
4170336,abdera,commits,2007-01-05 17:58:47,7,<20070105175847.C76E31A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,106.0,493105.0,svn commit: r493105 - in /incubator/abdera/jav...,svn,james m. snell,False,True,James Snell
4170337,abdera,commits,2007-01-05 17:59:41,7,<20070105175941.4FD541A981C@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,1.0,493106.0,svn commit: r493106 - /incubator/abdera/java/...,svn,james m. snell,False,True,James Snell
4170338,abdera,commits,2007-01-05 18:04:21,7,<20070105180421.B9EE71A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,18.0,493109.0,svn commit: r493109 - /incubator/abdera/java/...,svn,james m. snell,False,True,James Snell
4170339,abdera,commits,2007-01-05 18:12:15,7,<20070105181215.AD4221A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,16.0,493112.0,svn commit: r493112 - /incubator/abdera/java/...,svn,james m. snell,False,True,James Snell
4170340,abdera,commits,2007-01-05 19:34:39,7,<20070105193439.784C01A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,2.0,493154.0,svn commit: r493154 - in /incubator/abdera/jav...,svn,james m. snell,False,True,James Snell
4170341,abdera,commits,2007-01-05 19:34:39,7,<20070105193439.784C01A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,2.0,493154.0,svn commit: r493154 - in /incubator/abdera/jav...,svn,james m. snell,False,True,James Snell
4170342,abdera,commits,2007-01-05 19:34:39,7,<20070105193439.784C01A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,57.0,493154.0,svn commit: r493154 - in /incubator/abdera/jav...,svn,james m. snell,False,True,James Snell
4170343,abdera,commits,2007-01-05 19:55:15,7,<20070105195515.D5EEC1A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/parser/src/main/ja...,9.0,493164.0,svn commit: r493164 - /incubator/abdera/java/...,svn,james m. snell,False,True,James Snell
4170344,abdera,commits,2007-01-05 20:25:54,7,<20070105202554.54A501A981A@eris.apache.org>,jmsnell,jmsnell@apache.org,jmsnell,NaN,incubator/abdera/java/trunk/core/src/main/java...,40.0,493177.0,svn commit: r493177 - /incubator/abdera/java/...,svn,james m. snell,False,True,James Snell


In [22]:
print('grouping by author...')
projs = []
author_full = []
author_aliase = []
for project in tqdm(df):
	this_proj_df = df[project]
	this_proj_df = this_proj_df[this_proj_df['dealised_author_full_name'].notna()]
	author_df_dict = dict(tuple(this_proj_df.groupby(this_proj_df['dealised_author_full_name'])))

	for author in author_df_dict:		
		author_df = author_df_dict[author]
		this_author_aliases = author_df['author_name'].drop_duplicates().values
		author_aliase.extend(this_author_aliases)
		for i in range(len(this_author_aliases)):
			author_full.append(author)
			projs.append(project)
		if author_df.empty: continue
		file_path = to_path + '{}__{}.csv'.format(project, author)
		author_df.to_csv(file_path, index=False)
print('Commits Done.')

grouping by author...


100%|██████████| 263/263 [01:11<00:00,  3.69it/s]

Commits Done.


In [31]:
df_commiters = pd.DataFrame()

In [32]:
df_commiters['project_name'] = pd.Series(projs)
df_commiters['commiter_aliase'] = pd.Series(author_aliase)
df_commiters['commiter_full'] = pd.Series(author_full)

In [33]:
df_commiters.drop_duplicates()

,project_name,commiter_aliase,commiter_full
0,abdera,bcm,Brian Moseley
1,abdera,dandiep,Daniel Diephouse
2,abdera,calavera,David Calavera
3,abdera,eliast,Elias Torres
4,abdera,rooneg,Garrett Rooney
...,...,...,...
7525,zipkin,Raja Sundaram,Raja Sundaram
7526,zipkin,Sílvia Mur Blanch,Slvia Mur Blanch
7527,zipkin,tacigar,Tacigar
7528,zipkin,Zoltan Nagy,Zoltan Nagy


In [34]:
df_commiters.to_csv("./commiters_list.csv", index=False)

In [37]:
print('reading emails...')
df = pd.read_csv('../email_check/emails_preprocessed.csv')
print('grouping by project...')
df = dict(tuple(df.groupby(df['project_name'])))
to_path = '../byauthor_data/emails/'
if not os.path.exists(to_path):
	os.makedirs(to_path)

reading emails...
grouping by project...


In [38]:
df_commiters = pd.read_csv("./commiters_list.csv")
author_full = df_commiters["commiter_full"].values
author_aliase = df_commiters["commiter_aliase"].values

In [39]:
print('grouping by author...')
for project in tqdm(df):
	this_proj_df = df[project]
	this_proj_df = this_proj_df[this_proj_df['dealised_author_full_name'].notna()]
	author_df_dict = dict(tuple(this_proj_df.groupby(this_proj_df['dealised_author_full_name'])))

	for author in author_df_dict:	
		author_df = author_df_dict[author]
		#this_author_aliases = author_df['author_name'].drop_duplicates().values

		if author not in author_full:
			#for aliase in this_author_aliases:
			#	if aliase in author_aliase:
			#		author_full.append(author)
			#		author_aliase.append(aliase)
			#		df_commiters = df_commiters.append({"project_name": project, "commiter_aliase": aliase, "commiter_full": author})
			#		break
			continue
	
		if author_df.empty: continue
		file_path = to_path + '{}__{}.csv'.format(project, author)
		author_df.to_csv(file_path, index=False)
print('Commits Done.')

grouping by author...


100%|██████████| 274/274 [00:39<00:00,  6.94it/s]

Commits Done.


In [41]:
commiters_files = os.listdir(to_path)
from_path = '../byauthor_data/emails/'
all_emails = []
for file in commiters_files:
    this_df = pd.read_csv(from_path+file)
    all_emails.append(this_df)

result = pd.concat(all_emails)


In [42]:
result.to_csv("./commiters_emails.csv", index=False)

In [43]:
result.head()

,message_id,references,project_name,list_name,date,dealised_author_full_name,subject,body
0,<CA+FBdFQ7jRT+mr9Q-Ew=5k8Kc7zbeT3aMn5PoYLbTt9L...,<CAJjAHoMFvhT+xv21hh6r_Myxzk2vpRAhLLB_-K2eA8we...,superset,dev,2017-06-05 10:12:28,Ashutosh Chauhan,check -- -,yes list fri jun jim jagielski jim com write g...
1,<CA+FBdFRTx6k6EMebeb4jh_62Z4-LuU0a=tKAabBb2shv...,<CAHEEp7WxZztTjyZ0Tz+oPeHAPHPoPBZb71hoL4njtfg7...,superset,dev,2017-09-26 09:47:56,Ashutosh Chauhan,react apache superset,indeed great news ashutosh mon sep pm maxime b...
2,<JIRA.13130300.1515699055000.127061.1517866140...,<JIRA.13130300.1515699055000@Atlassian.JIRA> <...,superset,dev,2018-02-05 21:29:00,Ashutosh Chauhan,jira commented superset- send password creatio...,page com atlassian jira plugin system issuetab...
0,<BANLkTi=gMnmH146_FTiQH3ahQnnpeUp_8A@mail.gmai...,<BANLkTi=oq_QEwnwfwmcbrL8p9DdnCwObUw@mail.gmai...,flume,dev,2011-06-23 17:57:03,Aaron Kimball,plan moving mailing list apache,thu jun pm jonathan hsieh jon com write arg se...
0,<CAGfeOkdx-oTBnpeFhmmeTUZeSc_mofzO25jz20eVKOL+...,<CACAoQH2JT4uoOZGRe5dS=tUyQtG=VzFDqNcoEpvPb5RJ...,cordova,dev,2012-01-06 11:59:06,Dan Silivestru,doe anyone actually like jira email sent -dev ...,well fri jan gtanner com write get rid jira em...


In [3]:
print('reading emails...')
df = pd.read_csv('../email_check/emails_preprocessed.csv')

reading emails...


In [4]:
order = df['dealised_author_full_name'].value_counts()

In [5]:
order = pd.DataFrame(order).reset_index(drop=False)

In [6]:
order.columns = ['dealised_author_full_name', 'counts']

In [7]:
order

,dealised_author_full_name,counts
0,Reza Naghibi,64306
1,Confluence,6345
2,Tim Armstrong,5811
3,Guillaume Nodet,5398
4,Jonathan Ellis,4776
...,...,...
42187,Kiritomoe,1
42188,Guile,1
42189,Andreas Eberhfer,1
42190,Guodong,1


In [10]:
order['seq']= pd.Series(np.arange(1,len(order)+1))

In [4]:
#order.to_csv("./emailauthorseq.csv", index=False)
order = pd.read_csv("./emailauthorseq.csv")

In [5]:
areamap = dict(order[['dealised_author_full_name','seq']].values.tolist())

In [6]:
df['counts_seq'] = df['dealised_author_full_name'].map(areamap)

In [7]:
df.sort_values('counts_seq', ascending= True)

,message_id,references,project_name,list_name,date,dealised_author_full_name,subject,body,counts_seq
700288,<jo6vt5wai56krknfb4conmiy.1411650791458@email....,NaN,devicemap,dev,2014-09-25 09:13:12,Reza Naghibi,whatwg adding property navigator getting device,NaN,1
696738,<jo6vt5wai56krknfb4conmiy.1411650791458@email....,NaN,devicemap,dev,2014-09-25 09:13:12,Reza Naghibi,whatwg adding property navigator getting device,NaN,1
696737,<jo6vt5wai56krknfb4conmiy.1411650791458@email....,NaN,devicemap,dev,2014-09-25 09:13:12,Reza Naghibi,whatwg adding property navigator getting device,NaN,1
696736,<jo6vt5wai56krknfb4conmiy.1411650791458@email....,NaN,devicemap,dev,2014-09-25 09:13:12,Reza Naghibi,whatwg adding property navigator getting device,NaN,1
696735,<jo6vt5wai56krknfb4conmiy.1411650791458@email....,NaN,devicemap,dev,2014-09-25 09:13:12,Reza Naghibi,whatwg adding property navigator getting device,NaN,1
...,...,...,...,...,...,...,...,...,...
576331,<2019012400313928844635@foxmail.com>+40C8D2CBB...,None,dubbo,dev,2019-01-24 00:31:39,Kiritomoe,discus timer design dubbo,hi folk dubbo dubbo scheduledthreadpoolexecuto...,42188
1319754,<JIRA.13223030.1553162729000.100594.1553162760...,<JIRA.13223030.1553162729000@Atlassian.JIRA> <...,netbeans,commits,2019-03-21 10:06:00,Guile,jira created netbeans- focus lost insert imple...,guile create netbeans summary focus lose inser...,42189
1319772,<JIRA.13223206.1553205601000.104730.1553205602...,<JIRA.13223206.1553205601000@Atlassian.JIRA> <...,netbeans,commits,2019-03-21 22:00:02,Andreas Eberhfer,jira created netbeans- npe compile file gradle...,andreas eberh fer create netbeans summary npe ...,42190
576131,<CABK_SmKkdr8ZmDtthPPKqgNwUY_aWNuGCzNhNNKRpUHp...,<CAHuHCSMAQpzxSWR=BaYda1pE5EUZDOXcN9AG=2TZoX69...,dubbo,dev,2019-01-09 17:12:55,Guodong,issue,ok pls look ian luo ian luo com guodong mind f...,42191
